In [1]:
import h5py
from tqdm import tqdm 
import subprocess
import glob
import os 
import numpy as np
import pandas as pd
PLANCK_2018_Ho = 2.1816926176539463e-18 ### CGS
PLANCK_2018_OmegaMatter = 0.3158
PLANCK_2018_OmegaLambda = 1. - PLANCK_2018_OmegaMatter
PLANCK_2018_OmegaRadiation = 0.

In [2]:
folder_path = "data/O3_data_NRSurr_compare/"
events_files = glob.glob(os.path.join(folder_path, '*.h5'))


In [3]:
for event_file in tqdm(events_files): 
    
    with h5py.File(event_file,'r+') as file:    
            waveform = list(file.keys())[0]
            
    args = [f"{waveform}", f"{event_file}",
            "--Ho", f"{PLANCK_2018_Ho}", "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", "--OmegaRadiation",
            f"{PLANCK_2018_OmegaRadiation}", "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", "-o", f"{folder_path}"]
    subprocess.run(["python", "h52csv.py", *args]) 

 87%|████████▋ | 41/47 [02:31<00:21,  3.55s/it]Traceback (most recent call last):
  File "/Users/utkarsh/GitHub/SpectralBNS/h52csv.py", line 118, in <module>
    raise ValueError("The number of samples is less than 10000")
ValueError: The number of samples is less than 10000
 89%|████████▉ | 42/47 [02:31<00:12,  2.55s/it]

Please check the input file:
data/O3_data_NRSurr_compare/IGWN-GWTC3p0-v2-GW200220_061928_PEDataRelease_mixed_cosmo.h5.


100%|██████████| 47/47 [02:52<00:00,  3.66s/it]


In [4]:
 event_folder_name = folder_path

events = np.loadtxt(event_folder_name + "event-list.txt", dtype = str)
for i in tqdm(range(len(events))):
    path = event_folder_name + events[i]
    pe = pd.DataFrame(np.genfromtxt(path, delimiter=",",names=True))
    df = pe.sort_values(by="luminosity_distance")
    grad = np.gradient(df["luminosity_distance"], df["redshift"])
    mask = np.isnan(grad)
    grad[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), grad[~mask])
    d_l_arr = df["luminosity_distance"]
    z_arr = df["redshift"]
    np.savetxt(event_folder_name + f"dl_dz_H0_pe/{events[i]}.csv", np.array([d_l_arr, z_arr, grad]).T, header="d_l_arr, z_arr, grad", delimiter=",")

100%|██████████| 46/46 [00:07<00:00,  6.01it/s]
